Блок 1: Теория (LoRA vs QLoRA и что тюнить)
1. LoRA или QLoRA?
LoRA: Вы замораживаете модель и учите только маленькие адаптеры. Требует загрузки базы в 16-bit. На 20 Гб VRAM вы сможете тюнить так только модели размером 7B.
QLoRA: Вы загружаете базу в 4-bit (сжимаете), а адаптеры учите сверху. Это экономит огромное количество памяти.
Ваш выбор: Только QLoRA (позволит тюнить 14B на 20Гб карте).
2. Какие модули тюнить?
В трансформерах (архитектура LLM) есть слои внимания (Attention) и полносвязные слои (MLP).
Минимум: q_proj, v_proj (Query/Value в Attention). Это дает средний результат.
Максимум (рекомендуется): Тюнить все линейные слои (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj). Unsloth делает это автоматически одной настройкой. Это дает максимальный прирост ума модели.

In [1]:
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
# Конфигурация
max_seq_length = 2048 # Длина контекста при обучении. Чем больше, тем больше нужно VRAM.
dtype = None # Auto detection (bfloat16 если карта новая, float16 если старая)
load_in_4bit = True # Включаем QLoRA (обязательно для 20GB VRAM)

model_name = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit" 

# 1. Загружаем базу
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Навешиваем адаптеры LoRA
# Unsloth сам определит целевые модули (target_modules) для Qwen
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Ранг матрицы. 16 - стандарт, 64 - для сложных задач (ест больше памяти)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16, # Обычно равно r или r*2.
    lora_dropout = 0, # Для оптимизации памяти ставим 0
    bias = "none",    # Тоже для оптимизации
    use_gradient_checkpointing = "unsloth", # Критически важно для экономии VRAM!
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

print("Модель готова к обучению с QLoRA!")


ModuleNotFoundError: No module named 'torch'

In [2]:
from datasets import Dataset

# Пример данных для дообучения (RAG-стиль или саммаризация)
# Вам нужно подготовить список словарей
raw_data = [
    {
        "instruction": "Суммируй следующий текст.",
        "input": "Qwen2.5 - это новая модель, которая показывает отличные результаты в кодинге и математике...",
        "output": "Qwen2.5 - мощная модель для кода и математики."
    },
    {
        "instruction": "Ответь на вопрос по контексту.",
        "input": "Контекст: Компания Unsloth ускоряет обучение LLM.\nВопрос: Что делает Unsloth?",
        "output": "Компания Unsloth занимается ускорением обучения больших языковых моделей."
    }
    # ... добавьте сюда свои 100-1000 примеров
]

dataset = Dataset.from_list(raw_data)

# Шаблон промпта (должен совпадать с тем, как модель училась, или быть стандартным Alpaca)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Нужно добавить токен конца генерации, иначе модель не заткнется

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Формируем полный текст
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Применяем форматирование
dataset = dataset.map(formatting_prompts_func, batched = True,)

ModuleNotFoundError: No module named 'datasets'